In [0]:
%sh 
sudo apt-get install maven -y
mvn install -DgroupId=com.microsoft.azure -DartifactId=spark-mssql-connector_2.12_3.0 -Dversion=1.0.0-alpha

In [0]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import monotonically_increasing_id

parametros = PrcGeneral(id_solicitud = 3).obtenerParametros()
archivo_entrada = parametros['archivo_entrada']
fecha_proceso = parametros['fecha_proceso'].strftime('%Y-%m-%d') 
connection = PrcGeneral().databaseConnect()
cursor = connection.cursor()
tabla_destino = parametros['tabla_destino']
tabla_destino_2 = parametros['tabla_destino_2']

#Limpiando la tablas
#delete_rcc_saldos_smt = "DELETE t2 FROM " + tabla_destino + " t1 INNER JOIN " + tabla_destino_2 + " t2 ON (t1.id = t2.rcc) WHERE t1.FECHA_PROCESO = '"+fecha_proceso+"'"
#delete_rcc_smt = "DELETE FROM " + tabla_destino + " WHERE fecha_proceso = '" + fecha_proceso+"'"
#cursor.execute(delete_rcc_saldos_smt)
#cursor.execute(delete_rcc_smt)
#connection.commit()
#connection.close()

rcc_id_maximo_existente = 0
spark = SparkSession.builder.appName("Prueba de carga").getOrCreate()

# Carga
df = spark.read.text('/mnt/ereportinterfaz/inseir/'+archivo_entrada)
df_rcc_raw = df.filter(df.value.substr(1,1) == "1")
df_saldos_raw = df.filter(df.value.substr(1,1) == "2")

# SPLIT EN RCC
split_rcc = split(df_rcc_raw.value, '\|')
df_rcc = df_rcc_raw.withColumn('ID', monotonically_increasing_id() + rcc_id_maximo_existente)\
                   .withColumn('CODIGO_DEUDOR_SBS', split_rcc.getItem(0))\
                   .withColumn('FECHA_PROCESO', split_rcc.getItem(1))\
                   .withColumn('TIPO_DOCUMENTO_TRIBUTARIO', split_rcc.getItem(2))\
                   .withColumn('DOCUMENTO_TRIBUTARIO', split_rcc.getItem(3))\
                   .withColumn('TIPO_DOCUMENTO_IDENTIDAD', split_rcc.getItem(4))\
                   .withColumn('DOCUMENTO_IDENTIDAD', split_rcc.getItem(5))\
                   .withColumn('TIPO_PERSONA', split_rcc.getItem(6))\
                   .withColumn('CANTIDAD_EMPRESAS', split_rcc.getItem(7))\
                   .withColumn('DEUDA_CALIFICACION_0', split_rcc.getItem(8))\
                   .withColumn('DEUDA_CALIFICACION_1', split_rcc.getItem(9))\
                   .withColumn('DEUDA_CALIFICACION_2', split_rcc.getItem(10))\
                   .withColumn('DEUDA_CALIFICACION_3', split_rcc.getItem(11))\
                   .withColumn('DEUDA_CALIFICACION_4', split_rcc.getItem(12))\
                   .withColumn('APELLIDO_PATERNO_RAZON_SOCIAL', split_rcc.getItem(13))\
                   .withColumn('APELLIDO_MATERNO', split_rcc.getItem(14))\
                   .withColumn('APELLIDO_CASADA', split_rcc.getItem(15))\
                   .withColumn('PRIMER_NOMBRE', split_rcc.getItem(16))\
                   .withColumn('SEGUNDO_NOMBRE', split_rcc.getItem(16))
df_rcc = df_rcc.drop('value')

df_rcc_saldo = df_saldos_raw.select(df_saldos_raw.value.substr(2,11).alias('CODIGO_DEUDOR_SBS'),\
                                    df_saldos_raw.value.substr(12,16).alias('EMPRESA'),\
                                    df_saldos_raw.value.substr(17,17).alias('TIPO_CREDITO_SBS'),\
                                    df_saldos_raw.value.substr(18,31).alias('CUENTA_CONTABLE'),\
                                    df_saldos_raw.value.substr(32,35).alias('DIAS_ATRASO'),\
                                    df_saldos_raw.value.substr(36,50).alias('IMPORTE'),\
                                    df_saldos_raw.value.substr(36,50).alias('CLASIFICACION_DEUDOR'))
df_rcc_saldo = df_rcc_saldo.drop('value')

#df.show()
#df_rrc_saldo_consolidado = 
# SPLIT EN RCC SALDO





In [0]:
from pyspark.sql import SparkSession

parametros = PrcGeneral(id_solicitud = 17).obtenerParametros()
archivo_entrada = parametros['archivo_entrada']

spark = SparkSession.builder.appName("Prueba de carga").getOrCreate()
df = spark.read.text('/mnt/ereportinterfaz/inseir/'+archivo_entrada)
print(archivo_entrada)
url = "jdbc:sqlserver://auqui.database.windows.net;database=dbnormativo"
tabla = "TEMP_RCC"
usuario = "administrador"
clave = "Auqui$2020"
try:
  df.write.format("com.microsoft.sqlserver.jdbc.spark").mode("overwrite").option("truncate", "true").option("url", url).option("dbtable", tabla).option("user", usuario).option("password", clave).option("tableLock", "true").save()
except:
    raise
    

